In [1]:
import numpy as np

from scipy.constants import e, c

from scipy.constants import physical_constants
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3

In [2]:
from cpymad.madx import Madx

In [3]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False
madx.options.info = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.03.16 19:04:34      +
  ++++++++++++++++++++++++++++++++++++++++++++


# Thick linear lattice

Loading the SIS100 lattice sequence:

In [4]:
madx.call('./SIS100RING_220618.seq')

As usual, we power the three cold quadrupole families at `kqd`, `kqf` -- at the same time, the two warm quadrupoles `K1NL_S52QD11` and `K1NL_S52QD12` are linked by optimised factors to `kqd` and `kqf`.

The two degrees of freedom for the two transverse tunes given the entire lattice are hence `kqd` and `kqf`:

In [5]:
madx.input('''
kqd := -0.2798446835; ! 18.84, 18.73 tune
kqf := 0.2809756135;

K1NL_S00QD1D :=  kqd ;
K1NL_S00QD1F :=  kqf ;
K1NL_S00QD2F :=  kqf ;
K1NL_S52QD11 :=  1.0139780   *   kqd ;
K1NL_S52QD12 :=  1.0384325   *   kqf ;
''')

# # for other tune matching:

# madx.input('''
# match, sequence=SIS100RING;
# global, sequence=SIS100RING, q1=18.84, q2=18.73;
# vary, name=kqf, step=0.00001;
# vary, name=kqd, step=0.00001;
# lmdif, calls=500, tolerance=1.0e-10;
# endmatch;
# ''')

True

In [6]:
A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV

In [7]:
madx.command.beam(particle='ion', mass=A*nmass, charge=Q, energy=Etot)

True

All elements in the lattice for now:

In [8]:
list(np.unique([str(f.base_type).split(',')[0] for f in madx.sequence.sis100ring.elements]))

['collimator',
 'elseparator',
 'hmonitor',
 'kicker',
 'marker',
 'multipole',
 'quadrupole',
 'rfcavity',
 'sbend',
 'sextupole',
 'tkicker',
 'vmonitor']

Removing all elements from the thick lattice except for dipole magnets, main quadrupole magnets (cold+warm but no correctors) and rfcavities:

In [9]:
madx.input('''
!remove correctors
select, flag=seqedit, pattern=qs1j;
!remove all other elements
select, flag=seqedit, class=collimator;
select, flag=seqedit, class=kicker;
select, flag=seqedit, class=tkicker;
select, flag=seqedit, class=elseparator;
select, flag=seqedit, class=hmonitor;
select, flag=seqedit, class=multipole;
select, flag=seqedit, class=sextupole;
select, flag=seqedit, class=vmonitor;

seqedit, sequence=sis100ring;
    remove, element=selected;
    flatten;
endedit;

!remove remaining markers (besides $START which is required for MADX)
select, flag=seqedit, class=marker;
seqedit, sequence=SIS100RING;
    remove, element=selected;
    install, element=SIS100RING$START, s=0;
    flatten;
endedit;
''')

madx.use(sequence='sis100ring')

Only remaining elements:

In [10]:
list(np.unique([str(f.base_type).split(',')[0] for f in madx.sequence.sis100ring.elements]))

['marker', 'quadrupole', 'rfcavity', 'sbend']

The tune is still $18.84, 18.73$:

In [11]:
madx.twiss();

enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
            1083.6                 -0      0.00421952424        15.39459461 

                q1                dq1            betxmax              dxmax 
             18.84       -39.61634385         19.3794034        3.076634419 

             dxrms             xcomax             xcorms                 q2 
       1.384067113                  0                  0              18.73 

               dq2            betymax              dymax              dyrms 
      -39.72423421        20.69388134                  0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            synch_3            synch_4            synch_5 
                 0                  0                  0                  0 

            nflips 
          

In [12]:
madx.save(sequence='sis100ring', file='SIS100_linear_thick.seq')

True

# Thin lattice

In [13]:
madx.call('SIS100_linear_thick.seq')

In [14]:
madx.use(sequence='sis100ring')

In [15]:
assert madx.command.select(
    flag='MAKETHIN',
    class_='QUADRUPOLE',
    slice_='9',
)

assert madx.command.select(
    flag='MAKETHIN',
    class_='SBEND',
    slice_='9',
)

assert madx.command.makethin(
    makedipedge=True,
    style='teapot',
    sequence='sis100ring',
)

makethin: style chosen : teapot
makethin: slicing sequence : sis100ring


In [16]:
madx.use(sequence='sis100ring')

Also for the thin lattice, the tune remains very close to the originally set values $18.84, 18.73$:

In [17]:
madx.twiss();

enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
            1083.6                 -0     0.004219834221        15.39402917 

                q1                dq1            betxmax              dxmax 
       18.83923172       -39.80625205        19.95755085          3.1396633 

             dxrms             xcomax             xcorms                 q2 
       1.420816559                  0                  0        18.73013345 

               dq2            betymax              dymax              dyrms 
      -39.71349872        21.30558681                  0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            synch_3            synch_4            synch_5 
                 0                  0                  0                  0 

            nflips 
          

In [18]:
madx.save(sequence='sis100ring', file='SIS100_linear_thin.seq')

True